# Topic ideas

---

Group name: **EDART Collective**

Project owner: **Simon H.**

Semester: **WS 2025/26**

---

*Alle folgenden Topics basieren auf dem End-to-End Darts Analytics Vorhaben (FDI Rating) und nutzen die bereits geplanten Scraper, PostgreSQL-Pipeline sowie Gradio Deployment.*

## End-to-End Darts Analytics: FDI Forecasting

### Data source
- **Primary:** [Darts Orakel](https://dartsorakel.com) player pages + stats API (historic match aggregates, FDI baseline, order of merit, detailed per-leg KPIs). Data wird über einen eigenen Python-Scraper (Requests + BeautifulSoup + Playwright-Fallback) täglich aktualisiert.
- **Secondary enrichment:** PDC, Wikipedia, DartsRankings – für Turnier-Metadata, Preisgeldhistorie und World Ranking. Erfassung via REST-Endpoints bzw. HTML-Snapshots und Speicherung in einer self-hosted PostgreSQL-Instanz (Docker auf Hetzner).
- **Collection cadence:** Rolling 12M updates (Darts Orakel) + wöchentliche Full Refreshes der externen Quellen. Unser ETL (uv-managed Python jobs + cron) archiviert alle Snapshots versioniert.

### Data characterisitcs
- >2.9k Spieler, jeweils ~20 strukturierte Attribute (3-Dart Avg, Checkout %, 180s/Leg, First9 Avg, Order-of-Merit, FDI).
- Zeilen-Level: Spieler *und* Leg-basierte Zeitschienen (Granularität: Turnier/Match, Feature-Aggregation: last 12 months, trailing 30 matches, surface-level Hard vs Soft Facts).
- Spalten-Level: numerische KPIs, kategoriale Features (Nation, Tour-Card-Status), zeitliche Marker (Season, Form Window) plus engineered Features (Win %, log(Prize Money), Clutch Index).
- Datenvolumen nach ersten Crawls: ~3–5 MB CSV / ca. 50k Zeilen (Leg-Level) + 3k Zeilen (Player Master) → problemlos in Memory analysierbar, aber auch SQL/Parquet-tauglich.

### Research question
> Lässt sich das Darts FDI-Rating präzise prognostizieren, wenn Hard Facts (Score-, Checkout-, Order-of-Merit-KPIs) mit Soft Facts (Formkurven, Preisgeld-Historie, psychologische Proxies wie Clutch-Win%) kombiniert werden?
- **Response:** `fdi_rating` (kontinuierlich).
- **Predictors:** `three_dart_avg`, `first9_avg`, `checkout_pct`, `legs_won_pct`, `log_prize_money`, `win_pct`, `world_ranking`, `tour_card_years`, engineered momentum features.
- **Methodik:** Baseline OLS → Regularisierte Regressoren (Lasso/Ridge) → Tree Ensembles (Random Forest/XGBoost). Fokus auf Nonlinearitäten + Multikollinearität.

### Overview of data
- Persistenzpfad: `data/processed/player_stats_all.csv` (One-row-per-player wide table) + `data/raw/player_<id>_stats.csv` (Long format pro Spieler).
- Noch im Aufbau, aber erster Dump existiert (2 Spieler proof). Geplantes Schema: ca. 45 Spalten (Profil + Last-12-Month KPIs + API-Rank Informationen).
- Quick sanity checks laufen via Pandas Profiling & SQL (DuckDB) sobald Full Dump fertig ist.


In [1]:
import pandas as pd
from pathlib import Path

DATA_PATH = Path("../data/processed/player_stats_all.csv")

if DATA_PATH.exists():
    df = pd.read_csv(DATA_PATH)
    display(df.head())
    display(df.describe(include="all"))
else:
    print("Hinweis: Aggregierte CSV noch nicht vorhanden – Batch-Scraper läuft gerade.")

,player_id,player_name,country_code,birth_date,age,profile_total_earnings,profile_9_darters,profile_season_win_pct,profile_tour_card_years,profile_fdi_rating,...,last_12_months_pcnt_legs_won_throwing_first,last_12_months_pcnt_legs_won_throwing_second,last_12_months_171_180_s,last_12_months_140_s,last_12_months_131_140_s,last_12_months_functional_doubles_pcnt,api_rank,api_overall_stat,api_sum_field1,api_sum_field2
0,5403,Luke Littler,ENG,21/01/2007,18.0,2141879.0,8.0,80.0,1.0,1972.0,...,72.54,47.12,998.0,1268.0,1960.0,49.31,1,100.96,983928,29236
1,34,Luke Humphries,ENG,11/02/1995,30.0,3326675.0,6.0,68.0,7.0,1876.0,...,69.81,43.58,586.0,1131.0,1681.0,48.08,2,98.59,795358,24201
2,36,Stephen Bunting,ENG,09/04/1985,40.0,2000009.0,3.0,66.0,11.0,1819.0,...,70.12,40.42,679.0,1287.0,1684.0,46.30,3,98.04,851225,26046
3,20114,Josh Rock,NIR,13/04/2001,24.0,752150.0,6.0,68.0,3.0,1840.0,...,68.50,44.24,658.0,1004.0,1453.0,47.11,4,98.00,710292,21743
4,2641,Gian van Veen,NED,23/04/2002,23.0,499059.0,1.0,72.0,2.0,1868.0,...,71.95,45.78,595.0,1092.0,1653.0,55.40,5,97.92,846549,25937


,player_id,player_name,country_code,birth_date,age,profile_total_earnings,profile_9_darters,profile_season_win_pct,profile_tour_card_years,profile_fdi_rating,...,last_12_months_pcnt_legs_won_throwing_first,last_12_months_pcnt_legs_won_throwing_second,last_12_months_171_180_s,last_12_months_140_s,last_12_months_131_140_s,last_12_months_functional_doubles_pcnt,api_rank,api_overall_stat,api_sum_field1,api_sum_field2
count,2978.000000,2978,2276,276,262.000000,2.978000e+03,162.000000,2978.000000,249.000000,2477.000000,...,2978.000000,2978.000000,2755.000000,2949.000000,2972.000000,2978.000000,2978.000000,2978.000000,2.978000e+03,2978.000000
unique,NaN,2978,75,274,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Luke Littler,ENG,29/10/1993,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,1,573,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,27140.979181,NaN,NaN,NaN,41.255725,4.600531e+04,2.604938,46.524513,7.273092,1352.838514,...,55.579788,39.443526,39.110708,98.936589,140.056864,29.776377,1489.500000,69.086330,1.110692e+05,4393.288113
std,17183.041721,NaN,NaN,NaN,11.855430,3.673931e+05,3.240577,18.484709,5.024709,183.413271,...,13.262198,10.771679,80.616707,166.496145,240.654300,9.652255,859.818876,11.969559,1.378752e+05,4616.531169
min,1.000000,NaN,NaN,NaN,18.000000,0.000000e+00,1.000000,0.000000,0.000000,752.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,36.430000,1.426700e+04,1000.000000
25%,8204.500000,NaN,NaN,NaN,32.000000,0.000000e+00,1.000000,34.000000,3.000000,1228.000000,...,48.250000,33.330000,4.000000,16.000000,22.000000,22.680000,745.250000,60.310000,3.207075e+04,1505.250000
50%,27312.000000,NaN,NaN,NaN,40.000000,0.000000e+00,1.000000,50.000000,7.000000,1363.000000,...,58.330000,40.000000,11.000000,37.000000,50.000000,30.505000,1489.500000,69.970000,5.949400e+04,2658.500000
75%,45242.000000,NaN,NaN,NaN,51.000000,1.475000e+03,3.000000,60.000000,13.000000,1483.000000,...,64.710000,46.430000,32.000000,97.000000,133.250000,37.047500,2233.750000,78.010000,1.229380e+05,5204.250000


## Soft-Facts Impact: Clutch & Prize Dynamics

### Data source
- **Primary match logs:** Gleiche Darts-Orakel-APIs wie beim Hauptprojekt, aber granular auf Match-/Leg-Level (event=Major, Stage, Opponent Elo). Ergänzt durch `countup` Widgets, die Momentum-Serien liefern.
- **External signals:** Wikipedia/PDC für Turnier-Kategorien + Preisgeld, PDPA Releases für Tour-Card Informationen, ggf. Social-Sentiment (Twitter/Reddit) als optionaler Proxy.
- **Collection workflow:** Erweiterter Scraper speichert jede Begegnung als Fact-Tabelle in PostgreSQL (schema: `match_id`, `player_id`, `leg_index`, KPIs) inkl. historischen Snapshots.

### Data characterisitcs
- Beobachtungen: Jede Zeile = Spieler x Match x Leg, angereichert mit Drucksituationen (Checkout unter 40, Decider-Legs, Last-32 vs Final).
- Metriken: `win_pct_pressure`, `avg_checkout_under_pressure`, `decider_win_pct`, `log_prize_money_delta`, `travel_distance_proxy`, `rest_days`.
- Enthält sowohl numerische Indikatoren als auch kategoriell geclusterte Soft-Facts (z.B. Kontinente, Spieltage, Publikumstyp), wodurch Feature Engineering (one-hot/target encoding) nötig wird.

### Research question
> Wie stark beeinflussen Soft-Facts (Preisgeld-Schwankungen, Turnierphase, Travel/Rest-Pattern) die Clutch-Performance (Checkout in High-Pressure Legs) und lassen sich daraus Frühindikatoren für künftige Ranking-Sprünge ableiten?
- **Response:** `clutch_success_rate` (oder `decider_win_pct`).
- **Predictors:** `recent_prize_delta`, `stage_encoded`, `rest_days`, `opponent_rank_gap`, `audience_region`, `travel_distance_proxy`, `previous_leg_score_std`.
- Modellplanung: Panel-Regression + Mixed-Effects (Spieler-spezifische Random Effects) sowie Survival-Ansätze (Zeit bis nächster Ranking-Sprung).

### Overview of data
- Match-Level Tabellen werden als `data/interim/match_facts.parquet` abgelegt, bevor sie in PostgreSQL landen.
- Erste 2024er Major Events (Premier League, Worlds, World Matchplay) dienen als MVP-Korpus (~15k Legs).
- Explorative Auswertung via DuckDB/Pandas – Heatmaps für Drucksituationen, Rolling Averages zur Visualisierung von Momentum.


In [3]:
import pandas as pd
import duckdb
from pathlib import Path

MATCH_FACTS = Path("../data/interim/match_facts.parquet")

if MATCH_FACTS.exists():
    df_matches = pd.read_parquet(MATCH_FACTS)
    duckdb.query("SELECT stage, AVG(clutch_success) AS clutch, COUNT(*) AS n FROM df_matches GROUP BY stage ORDER BY clutch DESC LIMIT 10").show()
else:
    print("Match-Fact-Parquet noch nicht erzeugt – wird nach dem nächsten ETL-Durchlauf verfügbar sein.")

Match-Fact-Parquet noch nicht erzeugt – wird nach dem nächsten ETL-Durchlauf verfügbar sein.
